In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This is an updated version of a notebook contributed by [Fabian Hirschmann](https://github.com/fhirschmann) and [Rajesh Thallam](https://github.com/RajeshThallam).

<table align="left">

  <td>
    <a href="https://colab.sandbox.google.com/github/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_hpt_r_kernel.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_hpt_r_kernel.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_hpt_r_kernel.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                        
</table>

## Overview

This example demonstrates how to perform hyperparameter tuning of models written in R with `Vertex AI` using an R kernel -- such as in `Vertex AI Workbench Notebooks`.

### Dataset

The dataset used for this tutorial is [California Housing Dataset](https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html). The data contains information from the 1990 California census. The data set is publicly available from Google Cloud Storage at `gs://cloud-samples-data/ai-platform-unified/datasets/tabular/california-housing-tabular-regression.csv`. The dataset is used to train a Random Forest regressor to predict a median housing price, given a longitude and lattitude along with data from the corresponding census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).


### Objective

In this tutorial, you learn how to use `Vertex AI`, using an R kernel, for tuning hyperparameters of a R custom model.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Training`
- `Vertex AI Prediction`
- `Vertex AI Model` resource
- `Vertex AI Endpoint` resource

The steps performed include:

- Create a custom R training script
- Create a custom R deployment container.
- Perform hyperparameter tuning using `Vertex AI`.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* R
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with R and Python 3

The Cloud Storage guide to [Setting up a Python development environment](https://cloud.google.com/python/setup) and the [Jupyter installation guide](https://jupyter.org/install) provide detailed instructions for meeting these requirements. The following steps provide a condensed set of instructions:

1. [Install and initialize the SDK](https://cloud.google.com/sdk/docs/).

2. [Install R]().

3. [Install virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) and create a virtual environment that uses R.  Activate the virtual environment.

4. To install Jupyter, run `pip3 install jupyter` on the command-line in a terminal shell.

5. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

6. Open this notebook in the Jupyter Notebook Dashboard.


#### Define helper functions for this Notebook

First, you define some helper functions used throughout this tutorial.

- `sh`: Executes the specified command shell.
- `display_file`: Displays the contents of the specified file.

In [ ]:
library(glue)
library(IRdisplay)

sh <- function(cmd, args = c(), intern = FALSE) {
    if (is.null(args)) {
        cmd <- glue(cmd)
        s <- strsplit(cmd, " ")[[1]]
        cmd <- s[1]
        args <- s[2:length(s)]
    }
    ret <- system2(cmd, args, stdout = TRUE, stderr = TRUE)
    if ("errmsg" %in% attributes(attributes(ret))$names) cat(attr(ret, "errmsg"), "\n")
    if (intern) return(ret) else cat(paste(ret, collapse = "\n"))
}

display_file <- function(filename) {
    body <- sh(glue("pygmentize -g {filename} -f html -P full -O style=default"), intern = TRUE)
    IRdisplay::display_html(paste("<div style='text-align: left'>", paste(body, collapse="\n"), "</div>", collapse="\n"))
}

## Installation

Install the following packages to execute this notebook.

In [ ]:
required_packages <- c("reticulate", "glue", "httr")
install.packages(setdiff(required_packages, rownames(installed.packages())))

sh("pip install --upgrade google-cloud-aiplatform")

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com) and the [Artifact Registry API](https://console.cloud.google.com/flows/enableapi?apiid=artifactregistry.googleapis.com).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Have the project ID autodetected or enter it below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID <- "[your-project-id]"  # @param {type:"string"}

In [ ]:
# Get your Google Cloud project ID from gcloud
if (Sys.getenv("IS_TESTING") == "") {
    if (PROJECT_ID == "[your-project-id]") {
      PROJECT_ID <- sh("gcloud config list --format 'value(core.project)' 2>/dev/null", intern = TRUE)
    }
    cat("Project ID:", PROJECT_ID, "\n")
}

In [ ]:
sh("gcloud config set project {PROJECT_ID}")

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION <- "[your-region]"  # @param {type: "string"}

if (REGION == "[your-region]") {
    REGION <- "us-central1"
}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
TIMESTAMP = format(Sys.time(), "%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# The Google Cloud Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK <- file.exists("/opt/deeplearning/metadata/env_version")
IS_GOOGLE_COLAB <- (system("python -c 'import google.colab'") != 0)

GOOGLE_APPLICATION_CREDENTIALS <- ""  # @param {type:"string"}

# If on Vertex AI Workbench Notebooks, then don't execute this code
if (!IS_WORKBENCH_NOTEBOOK) {
    # Replace the string below with the path to your service account key and run this cell
    # to authenticate your GCP account.
    Sys.setenv("GOOGLE_APPLICATION_CREDENTIALS" = GOOGLE_APPLICATION_CREDENTIALS)
}

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex AI SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME <- "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI <- paste0("gs://", BUCKET_NAME)

In [ ]:
if (BUCKET_NAME == "" || BUCKET_NAME == "[your-bucket-name]") {
    BUCKET_NAME = paste0(PROJECT_ID, "aip-", TIMESTAMP)
    BUCKET_URI = paste0("gs://", BUCKET_NAME)
}

BUCKET_URI

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
sh("gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}")

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
sh("gsutil ls -al {BUCKET_URI}")

### Import libraries and define constants

Import and initialize the `reticulate` R package to interface with the Vertex AI SDK, which is written in Python.

In [ ]:
library(reticulate)
library(glue)
library(httr)
library(IRdisplay)

use_python(Sys.which("python3"))

aiplatform <- import("google.cloud.aiplatform")

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform$init(project = PROJECT_ID, location = REGION, staging_bucket = BUCKET_URI)

## Introduction to Google Artifact Registry

The `Google Artifact Registry` is a service for storing and managing artifacts in private repositories, including container images, Helm charts, and language packages. It is the recommended container image registry for Google Cloud.

Learn more about [Quick start for Docker](https://cloud.google.com/artifact-registry/docs/docker/quickstart)

### Enable Artifact Registry API

First, you must enable the Artifact Registry API service for your project.

Learn more about [Enabling service](https://cloud.google.com/artifact-registry/docs/enable-service).

In [ ]:
sh("gcloud services enable artifactregistry.googleapis.com")

## Create a private Docker repository

Your first step is to create your own Docker repository in Google Artifact Registry.

1. Run the `gcloud artifacts repositories create` command to create a new Docker repository with your region with the description "docker repository".

2. Run the `gcloud artifacts repositories list` command to verify that your repository was created.

In [ ]:
PRIVATE_REPO <- "my-docker-repo"

sh("gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description=\"Docker repository\"")

sh("gcloud artifacts repositories list")

### Configure authentication to your private repo

Before you push or pull container images, configure Docker to use the `gcloud` command-line tool to authenticate requests to `Artifact Registry` for your region.

In [ ]:
sh("gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet")

### Create Dockerfile

The docker file for your custom container is built on top of the Deep Learning container -- the same container that is also used for Vertex AI Workbench. In addition, you add an R script for model training.

In [ ]:
IMAGE_NAME <- "vertex-r"  # @param {type:"string"}
IMAGE_TAG <- "latest"  # @param {type:"string"}
IMAGE_URI <- glue("{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE_NAME}:{IMAGE_TAG}")

dir.create("src", showWarnings = FALSE)

In [ ]:
Dockerfile = cat("
# filename: Dockerfile - container specifications for using R in Vertex AI
FROM gcr.io/deeplearning-platform-release/r-cpu.4-1:latest

WORKDIR /root

COPY train.R /root/train.R

# Install Fortran
RUN apt-get update
RUN apt-get install gfortran -yy

# Install R packages
RUN Rscript -e \"install.packages('plumber')\"
RUN Rscript -e \"install.packages('argparser')\"
RUN Rscript -e \"install.packages('gbm')\"
RUN Rscript -e \"install.packages('caret')\"
RUN Rscript -e \"install.packages('reticulate')\"

RUN pip install cloudml-hypertune

EXPOSE 8080
", file = "src/Dockerfile")

display_file("src/Dockerfile")

### Create the training script

Next, create the file `train.R`, which is used to train your R model. The script trains a `gbm` model on the California Housing dataset. Vertex AI sets environment variables that you can utilize, and the hyperparameters for each trial are passed as command line arguments. The trained model artifacts are then stored to your Cloud Storage bucket. The results of your training script are communicated back to Vertex AI using the [hypertune](https://github.com/GoogleCloudPlatform/cloudml-hypertune) package, which stores a JSON file to `/tmp/hypertune/output.metrics`. Vertex AI uses this information to come up with a hyperparameter configuration for the next trial, and to assess which trial was responsible for the best overall result.

In [ ]:
cat('
#!/usr/bin/env Rscript
# filename: train.R - perform hyperparamter tuning on a boosted tree model using Vertex AI

Sys.getenv()
library(tidyverse)
library(data.table)
library(argparser)
library(jsonlite)
library(reticulate)
library(caret)

# The GCP Project ID
project_id <- Sys.getenv("CLOUD_ML_PROJECT_ID")

# The GCP Region
location <- Sys.getenv("CLOUD_ML_REGION")

# The Cloud Storage URI to upload the trained model artifact to
model_dir <- Sys.getenv("AIP_MODEL_DIR")

# The trial ID
trial_id <- Sys.getenv("CLOUD_ML_TRIAL_ID", 0)

# The JSON file to save metric results to
#metric_file <- Sys.getenv("CLOUD_ML_HP_METRIC_FILE")
metric_file <- "/var/hypertune/output.metrics"

# Hyperparameters for this trial
p <- arg_parser("California Housing Model") %>%
     add_argument("--n.trees", default = "100", help = "number of trees to fit", type = "integer") %>%
     add_argument("--interaction.depth", default = 3, help = "maximum depth of each tree") %>%
     add_argument("--n.minobsinnode", default = 10, help = "minimun number of observations in terminal node") %>%
     add_argument("--shrinkage", default = 0.1, help = "learning rate") %>%
     add_argument("--data", help = "path to the training data in GCS")

dir.create("/tmp/hypertune")
argv <- parse_args(p, unlist(strsplit(commandArgs(trailingOnly = TRUE), "=")))

system2("gsutil", c("cp", argv$data, "./data.csv"))
data <- fread("data.csv")
print(data)

print("Starting Model Training")
tuneGrid <- expand.grid(
    interaction.depth = as.integer(argv$interaction.depth),
    n.trees = as.integer(argv$n.trees),
    n.minobsinnode = as.integer(argv$n.minobsinnode),
    shrinkage = as.numeric(0.1)
)
print(tuneGrid)
fitControl <- trainControl(method = "cv", number = 3)
set.seed(42)
fit <- train(median_house_value ~ .,
             method = "gbm",
             trControl = fitControl,
             tuneGrid = tuneGrid,
             metric = "MAE",
             data = data
)

mean_absolute_error <- mean(fit$resample$MAE)
cat(paste("mean absolute error:", mean_absolute_error, "\\n"))

hypertune <- import("hypertune")
hpt = hypertune$HyperTune()
hpt$report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag = "mean_absolute_error",
    metric_value = as.numeric(mean_absolute_error),
    global_step = 1000)

saveRDS(fit$finalModel, "gbm.rds")
system2("gsutil", c("cp", "gbm.rds", model_dir))
', file = "src/train.R")

display_file("src/train.R")

### Build the Docker container

Next, you build the Docker container image on Cloud Build -- the serverless CI/CD platform.

*Note:* Building the Docker container image may take 10 to 15 minutes.

In [ ]:
sh("cd src && gcloud builds submit --region={REGION} --tag={IMAGE_URI} --timeout=1h")

### Define and submit hyperparameter tuning job

Once your training application is containerized, you define the machine specifications for the tuning job. In this example, you use `n1-standard-4` instances.

In [ ]:
worker_pool_specs <- list(
    list(
        'machine_spec' = list(
            'accelerator_count' = as.integer(0),
            'machine_type' = 'n1-standard-4'
        ),
        'container_spec' = list(
            "image_uri" = IMAGE_URI,
            "command" = c("Rscript", "train.R"),
            "args" = list("--data", "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/california-housing-tabular-regression.csv")
        ),
        'replica_count' = 1
    )
)

This specification is then used in a `CustomJob`.

In [ ]:
MODEL_DIR <- glue("{BUCKET_URI}/aiplatform-custom-job-{TIMESTAMP}")
custom_job <- aiplatform$CustomJob(
    display_name = "california-custom-job",
    worker_pool_specs = worker_pool_specs,
    base_output_dir = MODEL_DIR
)

Hyperparameter tuning jobs search for the best combination of hyperparameters to optimize your metrics. Hyperparameter tuning jobs do this by running multiple trials of your training application with different sets of hyperparameters.

You can control the job in the following ways:

- `max_trial_count`: Decide how many trials you want to allow the service to run. Increasing the number of trials generally yields better results, but it is not always so. Usually, there is a point of diminishing returns after which additional trials have little or no effect on the accuracy. Before starting a job with a large number of trials, you may want to start with a small number of trials to gauge the effect your chosen hyperparameters have on your model's accuracy. To get the most out of hyperparameter tuning, you shouldn't set your maximum value lower than ten times the number of hyperparameters you use.
- `parallel_trial_count`: You can specify how many trials can run in parallel. Running parallel trials has the benefit of reducing the time the training job takes (real time—the total processing time required is not typically changed). However, running in parallel can reduce the effectiveness of the tuning job overall. That is because hyperparameter tuning uses the results of previous trials to inform the values to assign to the hyperparameters of subsequent trials. When running in parallel, some trials start without having the benefit of the results of any trials still running.

In addition, you also need to specify which hyperparameters to tune. There is little universal advice to give about how to choose which hyperparameters you should tune. If you have experience with the machine learning technique that you're using, you may have insight into how its hyperparameters behave. You may also be able to find advice from machine learning communities.

However you choose them, it's important to understand the implications. Every hyperparameter that you choose to tune has the potential to increase the number of trials required for a successful tuning job. When you run a hyperparameter tuning job on Vertex AI, the amount you are charged is based on the duration of the trials initiated by your hyperparameter tuning job. A careful choice of hyperparameters to tune can reduce the time and cost of your hyperparameter tuning job.

There are [several data types supported for Vertex AI Hyperparameter Tuning Jobs](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview#data-types).

In [ ]:
hpt_job <- aiplatform$HyperparameterTuningJob(
    display_name = "california-hpt-job",
    custom_job = custom_job,
    max_trial_count = as.integer(14),
    parallel_trial_count = as.integer(2),
    metric_spec = list(
        "mean_absolute_error" = "minimize"
    ),
    parameter_spec = list(
        "n.trees" = aiplatform$hyperparameter_tuning$IntegerParameterSpec(
            min = as.integer(10), max = as.integer(1000), scale = "linear"
        ),
        "interaction.depth" = aiplatform$hyperparameter_tuning$IntegerParameterSpec(
            min = as.integer(1), max = as.integer(10), scale = "linear"
        ),
        "n.minobsinnode" = aiplatform$hyperparameter_tuning$IntegerParameterSpec(
            min = as.integer(1), max = as.integer(20), scale = "linear"
        )
    )
)

### Execute the hyperparameter tuning job


To train tune model, you call the method `run()`.

In [ ]:
hpt_job$run()
hpt_job

You can now watch the progress live in the Vertex AI console. Once the job has finished, you can see the results for each trial.

Finally, to list all trials and their respective results, we can inspect `hpt_job$trials`.

In [ ]:
results <- lapply(hpt_job$trials,
    function(x) { c(as.integer(x$id), as.numeric(x$final_measurement$metrics[[0]]$value)) }
)
results <- as.data.frame(do.call(rbind, results))
colnames(results) <- c("id", "metric")

In [ ]:
results

And find the trial with the lowest error.

In [ ]:
best_trial <- results[results$metric == min(results$metric), ]$id

In [ ]:
hpt_job$trials[[best_trial]]

The model artifacts can be found in Cloud Storage.

In [ ]:
sh("gsutil ls -r {MODEL_DIR}/{best_trial}")

## Clean up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
delete_bucket <- FALSE

hpt_job$delete()

sh("rm -rf src")

# remove Docker image
sh("docker rmi {IMAGE_URI}")

if (delete_bucket == TRUE) sh("gsutil -m rm -r {BUCKET_URI}")